# Interpolation methods

When working with time-series, it is often important to interpolate the data in order to : 1) fill in missing data points due to sampling errors, 2) make all time-series the same length for comparison of data.  There are several ways to interpolate time-series: 1) stretch the signal to a certain fixed length as performed by scipy's interp1d, and 2) expand the signal by a integer multiple of the desired fixed length.  Both  ways give a similar result, however if you perfer to preserve the frequency of the signal precisely the 2nd method is best.

Below is a practice of using several interpolation functions!

<img src="main_splash.png" alt="Drawing" style="width: 500px;"/>

In [11]:
# Interpolation_methods0
# <script src="https://gist.github.com/j622amilah/04ac7b599d36960a1dc04870b3c60299.js"></script>
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd

from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Toy problem

In [12]:
# Interpolation_methods1
# <script src="https://gist.github.com/j622amilah/4d9c7264eb4ca4995b95843f77559a9f.js"></script>
N = 10
A1 = 2
A2 = 2
k = 1
x_shortSIG = np.multiply(range(0, int((1/0.3)*N)), 0.3)
print('length of x_shortSIG : ', len(x_shortSIG))
x_longSIG = np.multiply(range(0, int((1/0.05)*N)), 0.05)
print('length of x_longSIG : ', len(x_longSIG))
b = 0
c = 0

shortSIG = A1*(np.sin(((2*np.pi)/k)*(x_shortSIG - b))) + c
longSIG = A2*(np.sin(((2*np.pi)/k)*(x_longSIG - b))) + c

length of x_shortSIG :  33
length of x_longSIG :  200


## Scipy library

In [13]:
# Interpolation_methods2
# <script src="https://gist.github.com/j622amilah/4017d809fd7a0dc5f20e27f5fd93f12c.js"></script>
x = np.linspace(shortSIG[0], len(shortSIG), num=len(shortSIG), endpoint=True)
y = shortSIG

f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')

xnew = np.linspace(shortSIG[0], len(shortSIG), num=len(longSIG), endpoint=True)

siglong_interp1d_linear = f(xnew)
siglong_interp1d_cubic = f2(xnew)

## Handmade function

In [14]:
# Interpolation_methods3
# <script src="https://gist.github.com/j622amilah/ff5a7ca0b475e310a567e3243bc62631.js"></script>
def make_a_properlist(vec):
    
    import numpy as np
    
    out = []
    for i in range(len(vec)):
        out = out + [np.ravel(vec[i])]
    vecout = np.concatenate(out).ravel().tolist()
    
    return vecout


def linear_intercurrentpt_makeshortSIGlong_NOtime(shortSIG, longSIG):
    
    rr = len(shortSIG)
    ss = len(longSIG)
    
    q = ss/rr
    q_floor = int(np.floor(ss/rr))
    ts1 = 1
    
    y_stretch_temp = []
    z = q_floor
    
    for k in range(len(shortSIG)-1):
        # Find slope='rise'/'run' between two points : scalar
        # The 'run' is by 2 to count for each point, otherwise the height will be deformed
        sig_m_temp = (shortSIG[k+1] - shortSIG[k])/(q*2)
        
        # The expanded points per shortSIG point
        sig_x_temp = np.multiply(range( k, int((1/ts1)*(z)) ), ts1)
        # print('sig_x_temp : ', sig_x_temp)
        
        # Estimate b from y=mx+b
        sig_b_temp = shortSIG[k] - sig_m_temp*sig_x_temp  # vector
        # print('sig_b_temp : ', sig_b_temp)
        
        # Accumulate the lines between each data point in shortSIG
        # Estimate of y=mx+b with b=first value of estimated b
        y_stretch_temp = y_stretch_temp + [sig_m_temp*sig_x_temp + sig_b_temp[0]]

        z = z + 1

    y_stretch_final_temp = make_a_properlist(y_stretch_temp)
    
    # Baseline shift to zero
    y_stretch_final = [i - y_stretch_final_temp[0] for i in y_stretch_final_temp]
    
    return y_stretch_final

In [15]:
# Interpolation_methods4
# <script src="https://gist.github.com/j622amilah/4a86b1361fe06e7d2723b16a38786dbe.js"></script>
y_stretch_final = linear_intercurrentpt_makeshortSIGlong_NOtime(shortSIG, longSIG)
print('length of y_stretch_final : ', len(y_stretch_final))

length of y_stretch_final :  192


## Pandas library

In [16]:
# Interpolation_methods5
# <script src="https://gist.github.com/j622amilah/3b87e20ccf4a08c6e0a6478947b6d077.js"></script>
rr = len(shortSIG)
ss = len(longSIG)

q_floor = int(np.floor(ss/rr))
# print('q_floor : ', q_floor)

y_stretch_temp = []
c = 0
for k in range(len(longSIG)-q_floor):
    if k%q_floor == 0:
        y_stretch_temp.append(shortSIG[c])
        c = c + 1
    else:
        y_stretch_temp.append(np.nan)
        
print('length of y_stretch_temp : ', len(y_stretch_temp))

s1 = pd.Series(y_stretch_temp)

lin_intrp_pandas = s1.interpolate()  # linear interpolation

ply_intrp_pandas = s1.interpolate(method='polynomial', order=2)    # polynomial interpolation

length of y_stretch_temp :  194


# Compare Scipy, handmade, and Pandas

In [21]:
# Interpolation_methods6
# <script src="https://gist.github.com/j622amilah/8f022b0b8ada25447d61a08a09c55e0b.js"></script>
fig = go.Figure()
config = dict({'scrollZoom': True, 'displayModeBar': True, 'editable': True})

fig = make_subplots(rows=4, cols=1)

# Short signal
fig.append_trace(go.Scatter(x=list(range(len(shortSIG))), y=shortSIG, name='shortSIG', line = dict(color='blue', width=2, dash='solid')), row=1, col=1)

# Handmade function
fig.append_trace(go.Scatter(x=list(range(len(longSIG))), y=longSIG, name='longSIG', line = dict(color='red', width=2, dash='solid'), showlegend=False), row=2, col=1)
fig.append_trace(go.Scatter(x=list(range(len(y_stretch_final))), y=y_stretch_final, name='linear intrp', line = dict(color='orange', width=2, dash='solid'), showlegend=False), row=2, col=1)

# Scipy
fig.append_trace(go.Scatter(x=list(range(len(longSIG))), y=longSIG, name='longSIG', line = dict(color='red', width=2, dash='solid'), showlegend=True), row=3, col=1)
fig.append_trace(go.Scatter(x=list(range(len(siglong_interp1d_linear))), y=siglong_interp1d_linear, name='linear intrp', line = dict(color='orange', width=2, dash='solid'), showlegend=True), row=3, col=1)
fig.append_trace(go.Scatter(x=list(range(len(siglong_interp1d_cubic))), y=siglong_interp1d_cubic, name='poly intrp', line = dict(color='cyan', width=2, dash='solid'), showlegend=True), row=3, col=1)

# Pandas
fig.append_trace(go.Scatter(x=list(range(len(longSIG))), y=longSIG, name='longSIG', line = dict(color='red', width=2, dash='solid'), showlegend=False), row=4, col=1)
fig.append_trace(go.Scatter(x=list(range(len(lin_intrp_pandas))), y=lin_intrp_pandas, name='linear intrp', line = dict(color='orange', width=2, dash='solid'), showlegend=False), row=4, col=1)
fig.append_trace(go.Scatter(x=list(range(len(ply_intrp_pandas))), y=ply_intrp_pandas, name='poly intrp', line = dict(color='cyan', width=2, dash='solid'), showlegend=False), row=4, col=1)

fig.update_layout(title='Signals')
fig.show(config=config)

## Octave/Matlab library

In [9]:
# Interpolation_methods7
# <script src="https://gist.github.com/j622amilah/92dce6f086623c514d20a5303f963674.js"></script>
shortSIG_long_linear = interp1(x_shortSIG, shortSIG, x_longSIG);
shortSIG_long_poly = interp1(x_shortSIG, shortSIG, x_longSIG,'spline');

NameError: name 'interp1' is not defined

## Excel 

In [ ]:
# Interpolation_methods8
# <script src="https://gist.github.com/j622amilah/300732081e2e3e1e17a9195812f5c722.js"></script>
=FORECAST(shortSIG_sampled_as_longSIG_empty_values,x_shortSIG,shortSIG)